In [21]:
import redis, datetime
ldfs_cache = redis.StrictRedis()
print(ldfs_cache.dbsize(), datetime.datetime.utcnow().isoformat())

0 2016-04-04T00:45:23.691425


In [22]:
import json, rdflib, os
LUA_LOCATION = "E:/2016/linked-data-fragments/lib/"
LUA_SCRIPTS = dict()
def setup():
    for name in ["add_get_triple",
                 "triple_pattern_search",
                "get_triple"]:
        filepath = os.path.join(
            LUA_LOCATION, "{}.lua".format(name))
        print(filepath, os.path.exists(filepath))
        with open(filepath) as fo:
            lua_script = fo.read()
        sha1 = ldfs_cache.script_load(lua_script)
        LUA_SCRIPTS[name] = sha1
setup()

E:/2016/linked-data-fragments/lib/add_get_triple.lua True
E:/2016/linked-data-fragments/lib/triple_pattern_search.lua True
E:/2016/linked-data-fragments/lib/get_triple.lua True


In [23]:
def process_source(source):
    triples_processed = 0
    rec_graph = rdflib.Graph()
    rec_graph.parse(data=json.dumps(source), format='json-ld')
    for s,p,o in rec_graph:
        triples_processed += 1
        ldfs_cache.evalsha(LUA_SCRIPTS["add_get_triple"], 3, str(s), str(p), str(o))
    return triples_processed

def process_dpla_json(dpla_filepath):
    dpla_json = json.load(open(dpla_filepath, errors='ignore'))
    start = datetime.datetime.utcnow()
    total_triples = 0
    print("Started processing {} records at {}".format(len(dpla_json), start.isoformat()))
    for i,row in enumerate(dpla_json):
        total_triples += process_source(row['_source'])
        if not i%100 and i>0:
            print(".", end="")
        if not i%1000:
            print(i, end="")
        if not i%5000 and i > 0:
            print(" triples={} ".format(total_triples))
    end = datetime.datetime.utcnow()
    print("Finished processing at {}, total time {} mins. Records {} Triples {}".format(
        end.isoformat(),
        (end-start).seconds / 60.0,
        len(dpla_json),
        total_triples))
        

In [24]:
process_dpla_json("E:/2016/tmp/uiuc.json")

Started processing 18231 records at 2016-04-04T00:45:43.291844
0..........1000..........2000..........3000..........4000..........5000 triples=180970 
..........6000..........7000..........8000..........9000..........10000 triples=359397 
..........11000..........12000..........13000..........14000..........15000 triples=539682 
..........16000..........17000..........18000..Finished processing at 2016-04-04T01:51:54.969163, total time 66.18333333333334 mins. Records 18231 Triples 656382


In [25]:
ldfs_cache.dbsize()

607982

In [26]:
607982-539682

68300

In [28]:
print("Starting Getty size={}".format(ldfs_cache.dbsize()))

Starting Getty size=0


In [ ]:
process_dpla_json("E:/2016/tmp/getty.json")